In [ ]:
#Importing libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px


try:
    import apyori
except:
    %pip install apyori

from apyori import apriori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=7d8ff1d41b297739d318996391e5ec0fe9df470189cccca6c2f6cc056bf18456
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [ ]:
#Loading Dataset
df1 = pd.read_csv('/content/product.csv')
df1.head()
df2 = pd.read_csv('/content/transaction_data.csv')
df = df1.merge(df2,on=["PRODUCT_ID"])
df.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT,household_key,BASKET_ID,DAY,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,1228,29046618323,157,1,3.49,3313,0.0,2213,23,0.0,0.0
1,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,358,30707611686,247,1,3.49,3266,0.0,1211,36,0.0,0.0
2,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,325,33046710871,410,4,13.96,3191,0.0,1139,59,0.0,0.0
3,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,1675,30760265177,250,1,0.99,3235,0.0,936,36,0.0,0.0
4,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,,1032,33783848749,458,1,1.59,33904,0.0,2034,66,0.0,0.0


In [ ]:
#Any null values
df.isnull().any()

PRODUCT_ID              False
MANUFACTURER            False
DEPARTMENT              False
BRAND                   False
COMMODITY_DESC          False
SUB_COMMODITY_DESC      False
CURR_SIZE_OF_PRODUCT    False
household_key           False
BASKET_ID               False
DAY                     False
QUANTITY                False
SALES_VALUE             False
STORE_ID                False
RETAIL_DISC             False
TRANS_TIME              False
WEEK_NO                 False
COUPON_DISC             False
COUPON_MATCH_DISC       False
dtype: bool

In [ ]:
df1 = df.iloc[:50000] # Suggest 10,000 for better processing time

In [ ]:
df1.shape

(50000, 18)

In [ ]:
#Total Products
all_products = df1['SUB_COMMODITY_DESC'].unique()
print("Total products: {}".format(len(all_products)))

Total products: 1139


In [ ]:
#Top 10 frequently sold products
def ditribution_plot(x,y,name=None,xaxis=None,yaxis=None):
    fig = go.Figure([
        go.Bar(x=x, y=y)
    ])

    fig.update_layout(
        title_text=name,
        xaxis_title=xaxis,
        yaxis_title=yaxis
    )
    fig.show()

In [ ]:
x = df1['SUB_COMMODITY_DESC'].value_counts()
x = x.sort_values(ascending = False) 
x = x[:10]

ditribution_plot(x=x.index, y=x.values, yaxis="Count", xaxis="Products")

In [ ]:
#One-hot representation of products purchased
one_hot = pd.get_dummies(df1['SUB_COMMODITY_DESC'])
df1.drop('SUB_COMMODITY_DESC', inplace=True, axis=1)
df1 = df1.join(one_hot)
df1.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,CURR_SIZE_OF_PRODUCT,household_key,BASKET_ID,DAY,QUANTITY,...,WINDSHIELD,WINGS,WOMEN S GENERAL-MAGAZINE,WOMENS HAIR SPRAYS,WRITING INSTRUMENTS,YEAST: DRY,YELLOW SUMMER SQUASH,YOGURT,YOGURT MULTI-PACKS,YOGURT NOT MULTI-PACKS
0,25671,2,GROCERY,National,FRZN ICE,22 LB,1228,29046618323,157,1,...,0,0,0,0,0,0,0,0,0,0
1,25671,2,GROCERY,National,FRZN ICE,22 LB,358,30707611686,247,1,...,0,0,0,0,0,0,0,0,0,0
2,25671,2,GROCERY,National,FRZN ICE,22 LB,325,33046710871,410,4,...,0,0,0,0,0,0,0,0,0,0
3,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,,1675,30760265177,250,1,...,0,0,0,0,0,0,0,0,0,0
4,26093,69,PASTRY,Private,BREAD,,1032,33783848749,458,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df1.shape, df.shape

((50000, 1156), (2595732, 18))

In [ ]:
records = df1.groupby(["BASKET_ID","DAY"])

In [ ]:
type(records)

pandas.core.groupby.generic.DataFrameGroupBy

In [ ]:
len(all_products[:])

1139

In [ ]:
records = df1.groupby(["BASKET_ID","DAY"]).head()

In [ ]:
records.shape

(44281, 1156)

In [ ]:
#Transactions
#Note: if a customer bought multiple products on same day, We will consider it one transaction
records = df1.groupby(["BASKET_ID","DAY"])[all_products[:]].apply(sum)
records = records.reset_index()[all_products]


In [ ]:
type(records)

pandas.core.frame.DataFrame

In [ ]:
records.shape

(33656, 1139)

In [ ]:
## Replacing non-zero values with product names
def get_Pnames(x):
    for product in all_products:
        if x[product] > 0:
            x[product] = product
    return x

records = records.apply(get_Pnames, axis=1)
records.head()

,ICE - CRUSHED/CUBED,NO SUBCOMMODITY DESCRIPTION,BREAD:ITALIAN/FRENCH,APPLE SAUCE,SPECIALTY COOKIES,SPICES & SEASONINGS,TRAY PACK/CHOC CHIP COOKIES,VITAMIN - MINERALS,SW GDS: SW ROLLS/DAN,HONEY,...,SEWING NOTIONS,SCHNAPPS,AIR CARE - CANDLES,LIQUEURS/SPECIALTIES (42 UNDER,HANES PANTIES,ANTIPERSPIRANTS ONLY (AEROSOL),DEODORANTS,TRADITIONAL RUM (42 UNDER PROO,FLAGS,FROZEN ENTREES
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
records.shape

(33656, 1139)

In [ ]:
print("total transactions: {}".format(len(records)))

total transactions: 33656


In [ ]:
records.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [ ]:
## Removing zeros
x = records.values
x = [sub[~(sub == 0)].tolist() for sub in x if sub[sub != 0].tolist()]
transactions = x

In [ ]:
print(len(transactions))

33656


In [ ]:

rules = apriori(transactions,min_support=0.00030,min_confidance=0.05,min_lift=70,min_length=2,target="rules")

In [ ]:
# Visualizing the top 5 results from Association mining results
count = 0
for item in rules:
    count += 1
    if count > 100:
        break
    items = list(item[0])
    print("Rule: " + items[0] + " -> " + items[1] + "\n")
    print("Support: " + str(item[1]) + "\n")
    print("Confidence: " + str(item[2][0].confidence) + "\n")
    print("Lift: " + str(item[2][0].lift) + "\n")
    print("=====================================\n")

Rule: BANANAS -> MEAT: TURKEY BULK

Support: 0.00032683622533872115

Confidence: 0.09649122807017542

Lift: 85.460757156048


Rule: BANANAS -> PIZZA/TRADITIONAL

Support: 0.00032683622533872115

Confidence: 0.09649122807017542

Lift: 111.98306110102841


Rule: BANANAS -> POTATOES RUSSET (BULK&BAG)

Support: 0.00041597337770382697

Confidence: 0.12280701754385964

Lift: 111.70791844476054


Rule: BANANAS -> POULTRY

Support: 0.0004456857618255289

Confidence: 0.13157894736842105

Lift: 77.6915974145891


Rule: BANANAS -> STRAWBERRIES

Support: 0.0004456857618255289

Confidence: 0.13157894736842105

Lift: 113.54925775978407


Rule: BANANAS -> TOMATOES VINE RIPE BULK

Support: 0.00032683622533872115

Confidence: 0.09649122807017542

Lift: 85.460757156048


Rule: BEANS GREEN: FS/WHL/CUT -> CORN

Support: 0.0003565486094604231

Confidence: 0.34285714285714286

Lift: 250.8521739130435


Rule: BETTER FOR YOU -> EGGS - X-LARGE

Support: 0.00038626099358212503

Confidence: 0.38235294117647056

